In [10]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [6]:
# Set your data directory and other parameters
data_dir = "D:/Curated X-Ray Dataset/"
batch_size = 32
img_size = (224, 224)  # ResNet50 input size

# Data augmentation and preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 7367 images belonging to 4 classes.
Found 1841 images belonging to 4 classes.


In [7]:

# Base model: ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# # Build the model on top of ResNet50
# model = models.Sequential()
# model.add(base_model)
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(4, activation='softmax'))  # Assuming 4 classes
    
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with softmax activation
output = layers.Dense(4, activation='softmax')(x)  # Assuming 4 classes

# Model without Added Layer
model = models.Model(base_model.input, output)

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [8]:

# Train the model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Evaluate the model
_, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")0

Epoch 1/10
231/231 [==============================] - 104s 436ms/step - loss: 1.5099 - accuracy: 0.5605 - val_loss: 1.2096 - val_accuracy: 0.3938
Epoch 2/10
231/231 [==============================] - 69s 299ms/step - loss: 0.6827 - accuracy: 0.7216 - val_loss: 0.6567 - val_accuracy: 0.7420
Epoch 3/10
231/231 [==============================] - 69s 299ms/step - loss: 0.7134 - accuracy: 0.7212 - val_loss: 0.7283 - val_accuracy: 0.7518
Epoch 4/10
231/231 [==============================] - 72s 310ms/step - loss: 0.7113 - accuracy: 0.7387 - val_loss: 1.2050 - val_accuracy: 0.7262
Epoch 5/10
231/231 [==============================] - 70s 301ms/step - loss: 0.6632 - accuracy: 0.7509 - val_loss: 0.6087 - val_accuracy: 0.7740
Epoch 6/10
231/231 [==============================] - 69s 300ms/step - loss: 0.6488 - accuracy: 0.7524 - val_loss: 0.6587 - val_accuracy: 0.7290
Epoch 7/10
231/231 [==============================] - 69s 300ms/step - loss: 0.8694 - accuracy: 0.7311 - val_loss: 0.4222 - val_a

In [11]:
# Evaluate the model
y_true = validation_generator.classes
y_pred_probs = model.predict(validation_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification report
class_labels = list(validation_generator.class_indices.keys())
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

58/58 [==============================] - 14s 245ms/step
Classification Report:
                     precision    recall  f1-score   support

           COVID-19       0.16      0.17      0.16       256
             Normal       0.35      0.35      0.35       654
Pneumonia-Bacterial       0.32      0.39      0.35       600
    Pneumonia-Viral       0.19      0.11      0.14       331

           accuracy                           0.29      1841
          macro avg       0.25      0.25      0.25      1841
       weighted avg       0.28      0.29      0.28      1841


Confusion Matrix:
[[ 43  91 105  17]
 [ 92 229 261  72]
 [ 85 227 231  57]
 [ 48 115 133  35]]
